In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
tf.__version__

2026-02-01 07:19:54.748772: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-01 07:19:54.789070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-01 07:19:55.631664: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


'2.20.0'

---

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
data = pd.read_csv("Iris.csv", index_col = "Id")
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
X = data.drop("Species", axis=1).to_numpy()
y = data[["Species"]].to_numpy()

In [6]:
# Preprocessing

scaler = StandardScaler()
onehot_encoder = OneHotEncoder(sparse_output=False)

enc_y = onehot_encoder.fit_transform(y)
scaled_X = scaler.fit_transform(X)

In [7]:
cat_array = onehot_encoder.categories_[0]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, enc_y, test_size=0.2, shuffle=True, stratify=enc_y, random_state=999
)

## Model Creation and Compile

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [10]:
inp_shape = X_train.shape[1]
num_classes = y_train.shape[1]

In [11]:
model = Sequential()

model.add(Input(shape= (inp_shape,))) #< Input Layer

model.add(Dense(64, activation="leaky_relu")) #< Hidden Layer
model.add(Dense(32, activation="leaky_relu")) #< Hidden Layer

model.add(Dense(num_classes, activation="softmax")) #< Output Layer

model.compile(
    optimizer="Adam", loss="categorical_crossentropy" , metrics=["accuracy"]
)

I0000 00:00:1769909696.776048   39907 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4095 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499 (9.76 KB)

 Trainable params: 2,499 (9.76 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X_train, y_train, epochs=25, batch_size=32, verbose=1
)

Epoch 1/25


2026-02-01 07:19:57.788849: I external/local_xla/xla/service/service.cc:163] XLA service 0x751bf4005590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-02-01 07:19:57.788860: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2026-02-01 07:19:57.802260: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-02-01 07:19:57.871402: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91801


1/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5938 - loss: 1.0027

I0000 00:00:1769909698.729743   40031 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 340ms/step - accuracy: 0.6250 - loss: 0.9760
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6583 - loss: 0.8678 
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6667 - loss: 0.7766 
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6667 - loss: 0.7015 
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6917 - loss: 0.6390 
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7333 - loss: 0.5882 
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7750 - loss: 0.5467 
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8083 - loss: 0.5110 
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8250 - loss: 0.4813 
Epoch 10/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8250 - loss: 0.4560 
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8333 - loss: 0.4348 
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8333 - loss: 0.4147 
Epoch 13/25

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
accuracy

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step - accuracy: 0.9000 - loss: 0.2569


0.8999999761581421

In [15]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [17]:
confusion_matrix(y_true, y_pred)

array([[10,  0,  0],
       [ 0,  8,  2],
       [ 0,  1,  9]])

In [18]:
accuracy_score(y_true, y_pred)

0.9

In [19]:
f1_score(y_true, y_pred, average="macro")

0.899749373433584

In [20]:
precision_score(y_true, y_pred, average="macro")

0.9023569023569024

In [21]:
recall_score(y_true, y_pred, average="macro")

0.9

In [39]:
np.argmax(model.predict(X_test[0:1]), axis =1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([2])

## Packaging a model

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,499 (29.30 KB)

 Trainable params: 2,499 (9.76 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,000 (19.54 KB)

In [25]:
model.save("iris_classification.keras")

In [23]:
import pickle

In [26]:
to_package = {
    "scaler" : scaler,
    "encoder" : onehot_encoder,
    "model": model
}

with open("iris_classifier.pkl","wb") as file:
    pickle.dump(to_package, file)